In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import time
import random
import cv2
import pandas as pd
from keras import layers, models, optimizers, losses, metrics, Sequential
from keras.layers import Dense, Conv2D, Flatten , Dropout, MaxPooling2D

In [29]:
# Generate our dataset for suits using the labeled images in the suit_images directory
# We will use the keras ImageDataGenerator to generate our dataset

suit_train = tf.keras.utils.image_dataset_from_directory("suit_images", labels="inferred", label_mode="categorical", color_mode="grayscale", batch_size=32, image_size=(100, 100), shuffle=True, seed=123, interpolation="bilinear", follow_links=False)
suit_validate = tf.keras.utils.image_dataset_from_directory("suit_images", labels="inferred", label_mode="categorical", color_mode="grayscale", batch_size=32, image_size=(100, 100), shuffle=True, seed=222, validation_split=0.8, subset="validation", interpolation="bilinear", follow_links=False)

print(suit_train.class_names)
print("Sizes: ", suit_train.cardinality(), suit_validate.cardinality())

print("Suit Train Shape: ", suit_train.element_spec)




Found 154 files belonging to 4 classes.
Found 154 files belonging to 4 classes.
Using 123 files for validation.
['clubs', 'diamonds', 'hearts', 'spades']
Sizes:  tf.Tensor(5, shape=(), dtype=int64) tf.Tensor(4, shape=(), dtype=int64)
Suit Train Shape:  (TensorSpec(shape=(None, 100, 100, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))


In [30]:
# Lets make a network
# We will use a convolutional neural network

# First we will define our model
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(100,100,1)))
model.add(MaxPooling2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(4, activation="softmax"))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 100, 100, 32)      320       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 50, 50, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 50, 50, 32)        9248      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 25, 25, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 25, 25, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 12, 12, 64)      

In [31]:
# Train the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(suit_train, epochs=20, validation_data=suit_validate)

Epoch 1/20
5/5 [==============================] - 1s 203ms/step - loss: 42.9459 - accuracy: 0.2403 - val_loss: 13.4958 - val_accuracy: 0.2846
Epoch 2/20
5/5 [==============================] - 1s 172ms/step - loss: 4.2764 - accuracy: 0.3442 - val_loss: 1.4598 - val_accuracy: 0.4634
Epoch 3/20
5/5 [==============================] - 1s 158ms/step - loss: 1.5341 - accuracy: 0.3831 - val_loss: 1.1303 - val_accuracy: 0.3902
Epoch 4/20
5/5 [==============================] - 1s 155ms/step - loss: 1.0938 - accuracy: 0.4870 - val_loss: 0.9685 - val_accuracy: 0.5772
Epoch 5/20
5/5 [==============================] - 1s 152ms/step - loss: 0.9476 - accuracy: 0.6364 - val_loss: 0.7331 - val_accuracy: 0.8211
Epoch 6/20
5/5 [==============================] - 1s 149ms/step - loss: 0.7196 - accuracy: 0.6883 - val_loss: 0.5043 - val_accuracy: 0.8293
Epoch 7/20
5/5 [==============================] - 1s 155ms/step - loss: 0.5457 - accuracy: 0.8117 - val_loss: 0.3159 - val_accuracy: 0.9593
Epoch 8/20
5/5 [==

In [32]:
# Save the model
model.save("suit_model.h5")

In [33]:
# Lets test the model

# Load the model in
model = tf.keras.models.load_model("suit_model.h5")

# Load the test image
test_image = cv2.imread("suit_images/hearts/hearts_10_suit_1680993207.517387.png", cv2.IMREAD_GRAYSCALE)
test_image = cv2.resize(test_image, (100, 100))
test_image = np.array(test_image)
test_image = test_image.reshape(1, 100, 100, 1)

# Make a prediction
prediction = model.predict(test_image)
print(prediction)
class_index = np.argmax(prediction)
print(class_index)
print(suit_train.class_names[class_index])



[[1.4726511e-04 2.9405509e-03 9.9678600e-01 1.2623395e-04]]
2
hearts


In [34]:
# Run through all of the images in the suit_images/hearts directory
# and make a prediction for each one
# Then print out the accuracy

# Get a list of all of the files in the directory using pathlib
path = pathlib.Path("suit_images/diamonds")

# Get a list of all of the files in the directory
files = list(path.glob("*.png"))

# Now lets sort the files by the time they were created
files.sort(key=lambda x: x.stat().st_ctime)

for file in files:
    # Now lets make a prediction for each image
    test_image = cv2.imread(file.as_posix(), cv2.IMREAD_GRAYSCALE)
    test_image = np.array(test_image)
    test_image = test_image.reshape(1, 100, 100, 1)

    # Make a prediction
    prediction = model.predict(test_image)
    class_index = np.argmax(prediction)
    c = suit_train.class_names[class_index]

    if c == "diamonds":
        pass
    else:
        print("Incorrect")
        cv2.imshow("Image", cv2.imread(file.as_posix(), cv2.IMREAD_GRAYSCALE))
        cv2.waitKey(0)


